# Import Packages

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 500)

# Read Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

PATH = '/content/drive/MyDrive/Citadel Datathon/Data/Ziao/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_raw = pd.read_csv(PATH+'data_raw.csv')
df_bid = pd.read_csv(PATH+'bidding.csv')

In [ ]:
df_bid = df_bid[['match_id','match_result','match_result_rel']]
df_raw = pd.merge(df_raw, df_bid, on=['match_id'])

In [ ]:
df_raw.date = pd.to_datetime(df_raw.date)

In [ ]:
# Sort by date and match first to split
df_raw = df_raw.sort_values(by=['date','match_id'],ascending=True)
train_df = df_raw.iloc[:int(df_raw.shape[0]*0.7),:]
test_df = df_raw.iloc[int(df_raw.shape[0]*0.7):,:]

# Categorical Features

In [ ]:
for c in train_df.columns:
  print(c)

match_id
season
stage
date
buildUpPlaySpeedDiff
buildUpPlayPassingDiff
chanceCreationPassingDiff
chanceCreationCrossingDiff
chanceCreationShootingDiff
defencePressureDiff
defenceAggressionDiff
max_height_diff
max_weight_diff
max_overall_rating_diff
max_potential_diff
max_crossing_diff
max_finishing_diff
max_heading_accuracy_diff
max_short_passing_diff
max_volleys_diff
max_dribbling_diff
max_curve_diff
max_free_kick_accuracy_diff
max_long_passing_diff
max_ball_control_diff
max_acceleration_diff
max_sprint_speed_diff
max_agility_diff
max_reactions_diff
max_balance_diff
max_shot_power_diff
max_jumping_diff
max_stamina_diff
max_strength_diff
max_long_shots_diff
max_aggression_diff
max_interceptions_diff
max_positioning_diff
max_vision_diff
max_penalties_diff
max_marking_diff
max_standing_tackle_diff
max_gk_diving_diff
max_gk_handling_diff
max_gk_kicking_diff
max_gk_positioning_diff
max_gk_reflexes_diff
max_age_diff
avg_height_diff
avg_weight_diff
avg_overall_rating_diff
avg_potential_diff


In [ ]:
id_cols = ['match_id','league_id','date','season']
target_cols = ['match_result','match_result_rel']
cat_cols = ["home_mode_preferred_foot", "home_mode_attacking_work_rate","home_mode_defensive_work_rate","away_mode_preferred_foot","away_mode_attacking_work_rate","away_mode_defensive_work_rate"]
num_cols = [i for i in train_df if i not in id_cols+target_cols+cat_cols and i != 'season']

In [ ]:
from scipy.stats import chi2_contingency

def rolling_chisq(train_df,col,diff=600):
  total_rows = train_df.shape[0]
  ps = pd.Series(dtype='float64')
  for i in range(diff,total_rows, diff):
    try:
      _, p, _, _ = chi2_contingency(pd.crosstab(train_df.loc[i-diff:i,'match_result'], train_df.loc[i-diff:i,col]))
      ps = ps.append(pd.Series([p]))
    except:
      pass
  return ps

In [ ]:
for c in cat_cols:
  res = rolling_chisq(train_df, c)
  print(c)
  print('Mean: %f, Std: %f'%(res.mean(),res.std()))


home_mode_preferred_foot
Mean: 0.373815, Std: 0.314106
home_mode_attacking_work_rate
Mean: 0.215629, Std: 0.308639
home_mode_defensive_work_rate
Mean: 0.361291, Std: 0.287089
away_mode_preferred_foot
Mean: 0.379431, Std: 0.276110
away_mode_attacking_work_rate
Mean: 0.243813, Std: 0.273285
away_mode_defensive_work_rate
Mean: 0.268873, Std: 0.299511


In [ ]:
# The categorical variables are not statistically related to target variable in a significant way and hence we can drop them

# Continuous Features

In [ ]:
from scipy.stats import spearmanr

def rolling_cor(train_df,col,diff=600):
  total_rows = train_df.shape[0]
  cors = pd.Series(dtype='float64')
  ps = pd.Series(dtype='float64')
  for i in range(diff,total_rows, diff):
    try:
      cor,p = spearmanr(train_df.loc[i-diff:i,'match_result_rel'], train_df.loc[i-diff:i,col])
      cors = cors.append(pd.Series([cor]))
      ps = ps.append(pd.Series([p]))
    except:
      pass
  return cors, ps
res = pd.DataFrame(columns=['Feature','Cor Mean', 'Cor Std', 'P Value Mean', 'P Value Std'])
for c in num_cols:
  cors, ps = rolling_cor(train_df, c)
  res = pd.concat([res, pd.DataFrame([[c, cors.mean(), cors.std(), ps.mean(), ps.std()]],columns=['Feature','Cor Mean', 'Cor Std', 'P Value Mean', 'P Value Std'])])


In [ ]:
res['Cor Mean'] = abs(res['Cor Mean'])

In [ ]:
res['Cor Mean Rank'] = res['Cor Mean'].rank(ascending=False)
res['Cor Std Rank'] = res['Cor Std'].rank(ascending=True)
res['P Value Mean Rank'] = res['P Value Mean'].rank(ascending=True)
res['P Value Std Rank'] = res['P Value Std'].rank(ascending=True)

In [ ]:
res['Total Rank'] = res['Cor Mean Rank'] + 0.5*res['Cor Std Rank'] + res['P Value Mean Rank'] + 0.5*res['P Value Std Rank']

In [ ]:
res = res.sort_values(by=['Total Rank']).reset_index(drop=True)

In [ ]:
res

,Feature,Cor Mean,Cor Std,P Value Mean,P Value Std,Cor Mean Rank,Cor Std Rank,P Value Mean Rank,P Value Std Rank,Total Rank
0,avg_potential_diff,0.365035,0.021879,1.003928e-06,3.755149e-06,2.0,2.0,7.0,7.0,13.5
1,avg_reactions_diff,0.356791,0.029571,4.911649e-07,1.771572e-06,3.0,17.0,3.0,2.0,15.5
2,max_overall_rating_diff,0.347166,0.028503,4.820443e-07,1.786471e-06,6.0,13.0,2.0,3.0,16.0
3,avg_overall_rating_diff,0.372563,0.029891,9.857007e-07,3.674611e-06,1.0,19.0,6.0,6.0,19.5
4,avg_short_passing_diff,0.348830,0.031098,1.511524e-06,5.478930e-06,5.0,21.0,8.0,9.0,28.0
5,avg_vision_diff,0.335127,0.033117,8.385252e-07,3.114340e-06,7.0,28.0,5.0,5.0,28.5
6,avg_ball_control_diff,0.353890,0.026537,6.657075e-06,2.485721e-05,4.0,9.0,16.0,16.0,32.5
7,avg_long_passing_diff,0.330469,0.034159,2.539599e-06,7.468046e-06,8.0,31.0,12.0,11.0,41.0
8,max_potential_diff,0.330025,0.027676,8.099472e-06,3.015202e-05,9.0,12.0,18.0,19.0,42.5
9,max_reactions_diff,0.312744,0.029776,7.364402e-06,2.643609e-05,12.0,18.0,17.0,17.0,46.5


In [ ]:
# 90 is a good cut off where the correlation starts to become lower than 0.1 and more statistical insignificant

In [ ]:
final_num_cols = list(res.loc[:91, 'Feature'])

# Check Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def get_vif(train_df,cols):
  temp_df = train_df[cols]
  vif_data = pd.DataFrame()
  vif_data["feature"] = cols
  vif_data["VIF"] = [variance_inflation_factor(temp_df.values, i)
                            for i in range(len(temp_df.columns))]
  return vif_data

In [ ]:
# Remove Variables until VIF value < 6
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
38,max_gk_positioning_diff,435.424942
45,std_gk_positioning_diff,364.771966
43,max_gk_handling_diff,361.934702
57,max_gk_reflexes_diff,349.487077
48,std_gk_handling_diff,300.591411


In [ ]:
final_num_cols.remove('max_gk_positioning_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
42,max_gk_handling_diff,306.992226
56,max_gk_reflexes_diff,275.673629
46,max_gk_diving_diff,262.512929
47,std_gk_handling_diff,255.322425
65,std_gk_reflexes_diff,224.218601


In [ ]:
final_num_cols.remove('max_gk_handling_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
45,max_gk_diving_diff,223.273418
55,max_gk_reflexes_diff,212.269422
50,std_gk_diving_diff,181.354594
64,std_gk_reflexes_diff,171.973224
3,avg_overall_rating_diff,48.376405


In [ ]:
final_num_cols.remove('max_gk_diving_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
54,max_gk_reflexes_diff,52.017334
3,avg_overall_rating_diff,48.374517
63,std_gk_reflexes_diff,46.669630
6,avg_ball_control_diff,19.467728
0,avg_potential_diff,17.832488


In [ ]:
final_num_cols.remove('max_gk_reflexes_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
3,avg_overall_rating_diff,48.315148
6,avg_ball_control_diff,19.466545
0,avg_potential_diff,17.825736
4,avg_short_passing_diff,15.657818
17,avg_dribbling_diff,11.332733


In [ ]:
final_num_cols.remove('avg_overall_rating_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
5,avg_ball_control_diff,18.887803
0,avg_potential_diff,16.644834
3,avg_short_passing_diff,15.447248
16,avg_dribbling_diff,11.332585
2,max_overall_rating_diff,9.773237


In [ ]:
final_num_cols.remove('avg_ball_control_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
0,avg_potential_diff,16.146953
3,avg_short_passing_diff,14.122345
2,max_overall_rating_diff,9.756790
15,avg_dribbling_diff,9.284425
5,avg_long_passing_diff,8.811253


In [ ]:
final_num_cols.remove('avg_potential_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
2,avg_short_passing_diff,13.929623
1,max_overall_rating_diff,9.750171
14,avg_dribbling_diff,9.164300
4,avg_long_passing_diff,8.807891
0,avg_reactions_diff,8.559242


In [ ]:
final_num_cols.remove('avg_short_passing_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
1,max_overall_rating_diff,9.672721
13,avg_dribbling_diff,8.862812
0,avg_reactions_diff,8.465050
23,avg_acceleration_diff,7.786318
49,avg_standing_tackle_diff,7.563813


In [ ]:
final_num_cols.remove('max_overall_rating_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
12,avg_dribbling_diff,8.859350
0,avg_reactions_diff,8.449178
22,avg_acceleration_diff,7.780935
48,avg_standing_tackle_diff,7.537973
1,avg_vision_diff,7.467041


In [ ]:
final_num_cols.remove('avg_dribbling_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
0,avg_reactions_diff,8.433902
21,avg_acceleration_diff,7.670399
47,avg_standing_tackle_diff,7.469410
1,avg_vision_diff,7.422251
2,avg_long_passing_diff,7.068585


In [ ]:
final_num_cols.remove('avg_reactions_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
20,avg_acceleration_diff,7.630766
46,avg_standing_tackle_diff,7.468762
0,avg_vision_diff,7.299376
1,avg_long_passing_diff,7.027594
29,avg_sprint_speed_diff,6.582443


In [ ]:
final_num_cols.remove('avg_acceleration_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
45,avg_standing_tackle_diff,7.456359
0,avg_vision_diff,7.298578
1,avg_long_passing_diff,7.026364
25,avg_long_shots_diff,6.441240
53,avg_marking_diff,5.893875


In [ ]:
final_num_cols.remove('avg_standing_tackle_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
0,avg_vision_diff,7.296972
1,avg_long_passing_diff,7.006143
25,avg_long_shots_diff,6.437588
4,max_ball_control_diff,5.851743
16,max_short_passing_diff,5.382263


In [ ]:
final_num_cols.remove('avg_vision_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
24,avg_long_shots_diff,6.435243
0,avg_long_passing_diff,6.365477
3,max_ball_control_diff,5.847814
15,max_short_passing_diff,5.380994
9,avg_crossing_diff,5.210146


In [ ]:
final_num_cols.remove('avg_long_shots_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
0,avg_long_passing_diff,6.181940
3,max_ball_control_diff,5.847237
15,max_short_passing_diff,5.374557
9,avg_crossing_diff,5.204616
4,avg_volleys_diff,4.950182


In [ ]:
final_num_cols.remove('avg_long_passing_diff')
vif_data = get_vif(train_df, final_num_cols)
vif_data.sort_values(by='VIF',ascending=False).head()

,feature,VIF
2,max_ball_control_diff,5.835758
14,max_short_passing_diff,5.363890
3,avg_volleys_diff,4.949882
38,std_gk_diving_diff,4.901499
12,avg_finishing_diff,4.858577


In [ ]:
final_num_cols.remove('max_ball_control_diff')
final_num_cols.remove('max_short_passing_diff')

# Output final Train and test

In [ ]:
final_cols = id_cols + final_num_cols+ target_cols
train_df[final_cols].to_csv(PATH + 'train.csv',index=False)
test_df[final_cols].to_csv(PATH + 'test.csv',index=False)

In [ ]:
train_df.shape

(11839, 131)

In [ ]:
train_df[final_cols].shape

(11839, 80)

In [ ]:
len(id_cols+target_cols)

6